In [1]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from model_code import *
from transformers import BartForConditionalGeneration
from transformers import BartTokenizer
from extract_keypoints import extract_keypoints_func

c:\Users\Wesley\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = 'saved_model'
model = load_custom_transformer_model(model_path)
model_path_bart = "bart_model"
model_bart = BartForConditionalGeneration.from_pretrained(model_path_bart)
model_bart = model_bart.to("cuda")

# Carregar o tokenizador
tokenizer_bart = BartTokenizer.from_pretrained("facebook/bart-large")

In [3]:
def predictions(kps):
    # SING2GLOSS
    kps = tf.convert_to_tensor(kps, dtype=tf.float32)
    kps = resize_pad(kps)
    kps = normalize(kps)
    kps = tf.expand_dims(kps, 0)
    pred = model.predict(kps, verbose=0)
    pred = beam_search(pred, tokenizer, beam_width=10)
    text = remove_consecutive_duplicates(pred[0][0])
    text = correcoes(text, file_path = 'annotations/words_with_hyphen.txt')
    print(text)

    # GLOSS2TEXT
    input_ids = tokenizer_bart.encode(text, return_tensors="pt", max_length=20, truncation=True)
    input_ids = input_ids.to("cuda")
    output_ids = model_bart.generate(input_ids, max_length=60)[0]
    decoded_output = tokenizer_bart.decode(output_ids, skip_special_tokens=True)
    print(decoded_output)

In [5]:
# Caminho para o vídeo
path = 'videos/01April_2010_Thursday_heute-6694.mp4'
kps = extract_keypoints_func(path)
text = predictions(kps)

LIEB ZUSCHAUER ABEND
liebe zuschauer guten abend
